<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-08-18 18:51:35


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['MCDOWELL-N','IBREALEST','CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS', 'UJJIVAN']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin

In [5]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='max')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_v20 = df_v20.append(df_tmp)
        df_v20 = pd.concat([df_v20, df_tmp])

    return df_v20.sort_index()

In [6]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [7]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [8]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'AV40N') | (df_v20_prospects['Criteria'] == 'AV40')]

In [9]:
df_final_v20 = pd.concat([df_v20_prospects_V40N.reset_index(), df_v20_prospects_200SMA.reset_index()])
df_final_v20.set_index("Date", inplace = True)
df_final_v20.sort_index()

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2006-06-09 00:00:00+05:30,BAJAJHIND.NS,43.27,266.0,258.118103,AVR
2006-08-02 00:00:00+05:30,BAJAJHIND.NS,20.80,289.0,270.067657,AVR
2007-02-02 00:00:00+05:30,NETWORK18.NS,27.78,657.0,364.143036,NaN
2007-02-07 00:00:00+05:30,NETWORK18.NS,24.26,659.0,423.114166,NaN
2007-02-13 00:00:00+05:30,TV18BRDCST.NS,23.07,104.0,87.111298,AVR
...,...,...,...,...,...
2024-01-03 00:00:00+05:30,5PAISA.NS,28.45,407.0,683.349976,AV40N
2024-04-04 00:00:00+05:30,VIPIND.NS,21.61,601.0,548.400024,AV40N
2024-04-04 00:00:00+05:30,VIPIND.NS,21.61,601.0,548.400024,AV40N


In [10]:
df_final_v20.sort_index().tail(20)

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2022-02-23 00:00:00+05:30,VIPIND.NS,25.66,491.0,644.989136,AV40N
2022-03-28 00:00:00+05:30,TATAELXSI.NS,20.52,5555.0,8833.672852,AV40N
2022-03-31 00:00:00+05:30,AAVAS.NS,21.73,2675.0,2569.449951,AVSP
2022-04-06 00:00:00+05:30,ALKYLAMINE.NS,20.53,3511.0,3308.266113,ASR
2022-04-06 00:00:00+05:30,VAIBHAVGBL.NS,29.36,585.0,493.475128,AVR
2022-05-09 00:00:00+05:30,CAMPUS.NS,24.04,379.0,378.899994,AVSP
2022-06-01 00:00:00+05:30,HAPPSTMNDS.NS,20.32,1188.0,973.478088,AAR
2022-07-19 00:00:00+05:30,VALIANTORG.NS,35.28,972.0,704.231384,AVR
2022-07-22 00:00:00+05:30,HAPPSTMNDS.NS,22.80,1086.0,983.168579,AAR


In [11]:
df_v20_prospects['Criteria'].value_counts()

,count
Criteria,
AVR,196
AVSP,49
AAR,46
AV40N,26
ASR,6
MGC,4
AV40,1
EXP50,1
